# TO DO


- размер окна уменьшим, посмотрим на квантиль абзацев по длине
- сделать опциональным, удаление больших описаний или разделение их на отдельные абзацы
- проверить на дубли датасет, разобраться с мусором
data[data['sentence'].duplicated(keep=False)].sort_values(['sentence'])
(tf_idf по схожим описаниям, построим попарные косинусные дистанции и посмотрим самых близких)


- вообще сделать ноутбук для обработки последующих текстов
- нужна отдельная обработка проекта по mittre - на его основе 
    - изучение основных разделителей атак
## эмбеддинги
    - изучение схожих описаний эмбеддингов (по tf/idf, по bert эмбеддингам посмотрим на схожие описания) 
    - тянуть url откуда предложения появились в датасет

## нужна интерпретация результатов

###
- оценить какие тактики сложно спрогнозировать и почему
    - для exfiltration, impact практически нет значимых слов, а для defense-evasion есть такие
    - у некоторых классов одинаковые значимые слова могут быть - impact exfiltration

- определить, каких тактик прям мало в выборках, из-за этого и ошибки
- min_df=10, max_df=0.1 для tf/idf на уровень слов

## разное
- [Потом почитать про аннотирование данных](https://github.com/center-for-threat-informed-defense/tram/wiki/Data-Annotation)


## Как можно улучшить классификацию тактик без значимых слов

- можно априор сделать равномерным, но тогда может увеличиться ошибочная классификация пустых предложений


## done

- вероятности классов подняли относительно пустого
- logloss возросла смотрел почему
- стемминг ухудшает - улучшает для нашего случая
- строил матрицу классов, чтобы у нас не перемешивались попадания
- может отдельный классификатор построить
- разбиение первичных описаний на разные абзацы, убрать ссылки в абзацах
    - есть списки как тут - https://attack.mitre.org/techniques/T1659/
- сделал countvectorizer + logreg лучше чем tf/idf
- 
- если валидация только на процедурных текстах, то качество выше из-за общих слов
- добавил бинарный классификатор на определение атака в тексте или нет